## Nutrient Demands



### Introduction



In our last project we used data to estimate systems of food demand
using different datasets.  An output from that project was as set of
`cfe.Result` objects; these bundle together both data and the results
from the demand system estimation, and can be used for prediction as
well.  

Here we&rsquo;ll explore some of the uses of the `cfe.Result` class, using
an instance created using data from the Indian ICRISAT data.

If you don&rsquo;t already have the latest version of the `cfe` package
installed, grab it, along with some dependencies:



In [1]:
!pip install -r requirements.txt

In [1]:
import pandas as pd
import cfe

### Data



We&rsquo;ll get data from two places.  First, basic data, including a food
 conversion table and recommended daily intakes table can be found in
 a google spreadsheet.



In [1]:
DataURL = 'https://docs.google.com/spreadsheets/d/13Ig5hZif-NSHtgkKRp_cEgKXk0lOsdUB2BAD6O_FnRo'
key = DataURL.split('/')[-1]

An instance `r` of `cfe.Result` can be made persistent with
 `r.to_dataset('my_result.ds')`, which saves the instance &ldquo;on disk&rdquo; in
 [NetCDF format](https://en.wikipedia.org/wiki/NetCDF), and can be loaded using `cfe.from_dataset`.  We use
 this method below to load data and demand system estimated for the
 Indian ICRISAT villages:



In [1]:
r = cfe.from_dataset('./Indian ICRISAT_small.ds')

Pick prices that were prevailing at some time and place; e.g.,



In [1]:
# Year 1975, village A
t,m = 75,1

p = r.prices.sel(t=t,m=m,drop=True)
phat = p.to_dataframe('i').dropna()
phat

So, we have demands for median household at that time and place



In [1]:
xbar = r.get_predicted_expenditures().sum('i')
median_x = xbar.where(xbar>0).sel(t=t,m=m).median('j') # Budget (median household)

xhat = r.get_predicted_expenditures().sel(t=t,m=m,drop=True).mean('j')

xhatdf = xhat.to_dataframe('i').dropna()

qhat = xhatdf.div(phat,axis=1).dropna()

qhat

Finally, define a function to change a single price in the vector $p$:



In [1]:
def my_prices(p0,p=p,i='Coconut'):
    p = p.copy()
    p.loc[i] = p0
    return p

### Mapping to Nutrients



We&rsquo;ve seen how to map prices and budgets into vectors of consumption
 quantities using `cfe.Result.demands`.  Next we want to think about
 how to map these into bundles of *nutrients*.  The information needed
 for the mapping comes from a &ldquo;Food Conversion Table&rdquo; (or database,
 such as the [USDA Nutrition DataBase](https://ndb.nal.usda.gov/ndb/doc/apilist/API-FOOD-REPORTV2.md)).   Here we&rsquo;ve adapted (with
 thanks to Alessandro Tarozzi of the Universitat Pompeu Fabra) a food
 conversion table developed by Gopalan et al (1980) to the foods
 reported in the Indian ICRISAT data:



In [1]:
from eep153_tools.sheets import read_sheets

fct = read_sheets(key,sheet='FCT').set_index('i')

We need the index of the Food Conversion Table (FCT) to match up with
 the index of the vector of quantities demanded.   To manage this we
 make use of the `align` method for `pd.DataFrames`:



In [1]:
# Create a new FCT and vector of consumption that only share rows in common:
fct0,c0 = fct.align(qhat,axis=0,join='inner')
print(fct0.index)

Now, since rows of `fct0` and `c0` match, we can obtain nutritional
 outcomes from the inner (or dot, or matrix) product of the transposed
 `fct0` and `c0`:



In [1]:
# The @ operator means matrix multiply
N = fct0.T@c0

# NB: ICRISAT data is *Annual* consumption
N

Of course, since we can compute the nutritional content of a vector of
 consumption goods `c0`, we can also use our demand functions to
 compute nutrition as a *function* of prices and budget.



In [1]:
def nutrient_demand(x,p):
    c = r.demands(x,p)
    fct0,c0 = fct.align(c,axis=0,join='inner')
    N = fct0.T@c0

    N = N.loc[~N.index.duplicated()]
    
    return N

With this `nutrient_demand` function in hand, we can see how nutrient
 outcomes vary with budget, given prices:



In [1]:
import numpy as np
import matplotlib.pyplot as plt

X = np.linspace(median_x/5,median_x*5,50)

UseNutrients = ['Protein','Calories','Iron','Calcium']

df = pd.concat({myx:np.log(nutrient_demand(myx,p))[UseNutrients] for myx in X},axis=1).T
ax = df.plot()

ax.set_xlabel('log budget')
ax.set_ylabel('log nutrient')

Now how does nutrition vary with prices?



In [1]:
USE_GOOD = 'Jowar/Sorghum'

ref_price = r.prices.sel(i=USE_GOOD,t=t,m=m,drop=True)

P = np.linspace(1,5,20).tolist()

ndf = pd.DataFrame({p0:np.log(nutrient_demand(median_x,my_prices(p0,i=USE_GOOD)))[UseNutrients] for p0 in P}).T

ax = ndf.plot()

ax.set_xlabel('log price')
ax.set_ylabel('log nutrient')